# Implied Volatility

## 1. Implied Volatility

Implied volatility (IV) is one of the most important parameter in options pricing. IV is determined by the current market price of option contracts on a particular underlying asset. IV is commonly
represented as a percentage that indicates the annualized expected one standard deviation range for the underlying asset implied from the option prices. 

IV $\sigma_{\text{imp}}$ is the volatility value $\sigma$ that makes the Black Scholes value of the option equal to the traded price of the option. In the Black-Scholes model, volatility is the only parameter that can’t be directly observed. All other parameters can be determined through market data and this parameter is determined by a numerical optimization technique given the Black-Scholes model.

In [2]:
# Data Manipulation
from numpy import *
from datetime import datetime
# Import blackscholes object
from optionpricer2 import BS
from tabulate import tabulate

## 2. Implied Volatility Formulation

The Black–Scholes equation describes the price of the option over time as:

$$
\frac{\partial V}{\partial t} + \frac{1}{2} \sigma^{2} S^{2} \frac{\partial ^{2}V}{\partial S^{2}} + rS \frac{\partial V}{\partial S} - rV = 0
$$

Solving the above equation, we know that the value of a call option for a non-dividend paying stock is:

$$
C = SN(d_{1}) − Ee^{−r(T−t)} N(d_{2})
$$

and, the corresponding put option price is:

$$
P = -SN(-d_{1}) + Ee^{−r(T−t)} N(-d_{2})
$$

Where:

$$
\begin{align*}
d_{1} &= \dfrac{\log\left(\frac{S}{E}\right)+ (r+ \frac 1 2 \sigma^{2})(T-t)}{\sigma \sqrt{T - t}}\\
\\
d_{2} &= \dfrac{\log\left(\frac{S}{E}\right)+ (r - \frac 1 2 \sigma^{2})(T-t)}{\sigma \sqrt{T - t}}\\
\\
N(x) &= \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{x} e^{-\frac 1 2 \phi^{2}} \: d\phi
\end{align*}
$$

We can look at the call and put equation as a function of the volatility parameter $\sigma$. Finding implied volatility thus requires solving the nonlinear problem $f(x) = 0$ where $x = \sigma$ given a starting estimate.

For call options we have:

$$
f(x) = SN(d_1) - Ee^{-rt} N(d_2) - C
$$

and for puts:

$$
f(x) = -SN(d_1) + Ee^{-rt}N(-d_2) - P
$$

To solve the function when $f(x) = 0$, numerical precedures like Bisection or Newton’s method are employed.


### 2.1 Newton Method

In [3]:
def newton_iv(className, spot, strike, rate, dte, callprice=None, putprice=None):
    x0 = 1              # Initial guess
    h = 0.001           # Step size
    tolerance = 1e-7    # 7-digit accuracy is desired
    epsilon = 1e-14     # Do not divide by a number smaller than this, some kind of error/floor
    maxiter = 200       # Maximum number of iterations to execute

    # Function whose root we are trying to find
    if callprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).callPrice - callprice
    if putprice:
        f = lambda x: eval(className)(spot, strike, rate, dte, x).putPrice - putprice

    for i in range(maxiter):
        y = f(x0)                                   # Starting with the initial guess
        yprime = (f(x0 + h) - f(x0 - h)) / (2 * h)  # Central difference, the derivative of the function

        if abs(yprime) < epsilon:                   # Stop if the denominator is too small
            break

        x1 = x0 - y / yprime                        # Perform Newton's computation

        if abs(x1 - x0) <= tolerance * abs(x1):     # Stop when the result is within the desired tolerance
            break

        x0 = x1                                     # Update x0 to start the process again

    return x1                                       # x1 is a solution within tolerance and the maximum number of iterations

In [4]:
# newton iv
newton_iv('BS',100,100,0.02,1,callprice=8)

0.17657213831399154

### 2.2 Bisection Method

In [5]:
def bisection_iv(className, spot, strike, rate, dte, callprice=None, putprice=None, high=500.0, low=0.0):
    # This is market price
    if callprice:
        price = callprice
    if putprice and not callprice:
        price = putprice

    tolerance = 1e-7

    for i in range(1000):
        mid = (high + low) / 2  # Calculate the midpoint (c = (a + b) / 2)
        
        if mid < tolerance:
            mid = tolerance

        if callprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).callPrice  # Black-Scholes price
        if putprice:
            estimate = eval(className)(spot, strike, rate, dte, mid).putPrice

        if round(estimate, 6) == price:
            break
        elif estimate > price:
            high = mid  # Replace c with b (b = c)
        elif estimate < price:
            low = mid  # Replace c with a (a = c)

    return mid

In [6]:
# bisection iv
bisection_iv('BS',100,100,0.02,1,callprice=8)

0.17657213902566582

## 3. BS Implied Volatility

Using Bisection Method to update our `optionpricer.py` class to calculate implied volatility

In [7]:
# Initialize option
option = BS(100, 100, 0.05, 1, 0.2, callprice=8)
header = ['Option Price', 'Delta', 'Gamma', 'Theta', 'Vega', 'Rho', 'IV']
table = [[option.callPrice, option.callDelta, option.gamma, option.callTheta,
          option.vega, option.callRho, option.impvol]]

print(tabulate(table, header))


  Option Price     Delta     Gamma       Theta     Vega       Rho        IV
--------------  --------  --------  ----------  -------  --------  --------
       10.4506  0.636831  0.018762  -0.0175727  0.37524  0.532325  0.133776
